In [1]:
import sys
sys.path.append('./')
import layout
import utils

import numpy as np
import pandas as pd
import random

In [2]:
items_df = pd.read_csv('../artifacts/top_100_sku.csv')
items_ordered_list = items_df['sku'].values.tolist()
set_items = set(items_df['sku'].values.tolist())

In [3]:
len(set_items)

100

In [4]:
top_25_items_list = pd.read_csv('../artifacts/top_items_together.txt')['sku'].tolist()
top_25_items_set = set(top_25_items_list)

In [5]:
len(top_25_items_set)

25

In [6]:
top_25_items_set - set_items

{901059, 901060, 901061, 901067, 901068, 901069, 902438, 911091, 911092}

In [7]:
together_sku_set = top_25_items_set.intersection(set_items)

In [8]:
ordered_list = list(together_sku_set) + list(set_items-together_sku_set)
len(ordered_list)

100

In [9]:
random_obj = layout.layout('../layout_files/row_layout_100.txt')
order_obj = layout.layout('../layout_files/row_layout_100.txt')
utils.randomly_assign_products_to_racks(order_obj, ordered_list)
random.shuffle(ordered_list)
utils.randomly_assign_products_to_racks(random_obj, ordered_list)

In [10]:
orders_df = pd.read_csv('../artifacts/order_2m_fixed.csv')

In [11]:
def func(row):
    split_row = row.split(',')
    split_row = [int(i) for i in split_row if i.isdigit()]
    return len(set(split_row)-set_items)
orders_df['len_other_skus'] = orders_df['sku'].apply(func)

In [12]:
orders_df.head()

,order_id,local_date,sku,sku_count,sku_id_count,len_other_skus
0,807636508,2022-07-01,"905851,901870,902459,902405,906075,912870,902450",7,7,4
1,806634633,2022-07-01,"900955,903142,903141,901839,903131,913044,9011...",9,9,8
2,806728678,2022-07-01,"900350,900351,915650,902129,916994,900510,9003...",9,9,6
3,807114392,2022-07-01,916174,1,1,1
4,806755515,2022-07-01,901078,1,1,1


In [13]:
orders_df[orders_df['len_other_skus'] == 1].shape

(11408, 6)

In [14]:
def new_func(row):
    split_row = row.split(',')
    split_row = [int(i) for i in split_row if i.isdigit()]
    return ','.join(list(str(i) for i in set(split_row).intersection(set_items)))
orders_df['new_skus'] = orders_df['sku'].apply(new_func)

In [15]:
orders_df['new_sku_len'] = orders_df['new_skus'].apply(lambda x: len(x.split(',')))

In [16]:
orders_df.shape

(69417, 8)

In [17]:
orders_df[orders_df['new_sku_len']>=2].shape

(26724, 8)

In [32]:
order_list = []
for _, row in orders_df[orders_df['new_sku_len']>=2].sample(n=1000).iterrows():
    order_list.append(row['new_skus'].split(','))
len_list = [len(l) for l in order_list]

In [33]:
print('avg order len =',np.mean(len_list))
print('std order len =',np.std(len_list))

avg order len = 3.464
std order len = 1.9418300646555042


In [34]:
order_list = [[int(i) for i in l] for l in order_list]

In [40]:
# calculating distance for order_obj
distance_list = []
for order in order_list:
    distance_list.append(utils.calc_distance(order_obj, (0,0), order))

In [41]:
print('avg order distance =',np.mean(distance_list))
print('std order  =',np.std(distance_list))
print('median order  =',np.median(distance_list))

avg order distance = 40.438
std order  = 23.395088287929156
median order  = 34.0


In [42]:
# calculating distance for random_obj
distance_list = []
for order in order_list:
    distance_list.append(utils.calc_distance(random_obj, (0,0), order))

In [43]:
print('avg random distance =',np.mean(distance_list))
print('std order  =',np.std(distance_list))
print('median order  =',np.median(distance_list))

avg random distance = 44.332
std order  = 25.3484866609429
median order  = 37.0
